<a href="https://colab.research.google.com/github/Somesh140/GPT3/blob/main/1_Dataset_creation_with_GPT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.8 MB/s eta 0:00:00


In [2]:
import os
import openai
import getpass

openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')



Enter OpenAI API key:··········


## Generate paraphrases with GPT-3

In [10]:
response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="I am a diverse paraphrasing expert. I will generate many random pairs of original and paraphrased sentences.\n\nHere are a few:\n\noriginal: Once, a group of frogs was roaming around the forest in search of water.\nparaphrase:  A gang of frogs was roaming about the woods in search of water once more.\n\noriginal:",
  temperature=0.7,
  max_tokens=100,
  top_p=0.9,
  frequency_penalty=0.5,
  presence_penalty=0.6,
  stop=["original"]
)



In [11]:
print(response)

{
  "choices": [
    {
      "finish_reason": null,
      "index": 0,
      "logprobs": null,
      "text": " The family had been given the best seats.\nparaphrase: The family had been gifted with the best seats."
    }
  ],
  "created": 1685621140,
  "id": "cmpl-7MbLgOX9CJcQRyT4sMjNIvBqYq3QM",
  "model": "davinci-instruct-beta",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 76,
    "total_tokens": 100
  }
}


In [12]:
output_string = response['choices'][0]['text'].strip()
output_modified = output_string.replace("paraphrase:","")
original_sentence, paraphrased_sentence = output_modified.split('\n')
original_sentence = original_sentence.strip()
paraphrased_sentence = paraphrased_sentence.strip()
print ("-----------------------------------------------------------------------")
print (original_sentence)
print (paraphrased_sentence)

-----------------------------------------------------------------------
The family had been given the best seats.
The family had been gifted with the best seats.


## Streamlit Dataset Creator UI

In [13]:
!pip install --quiet streamlit==1.1.0  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.4 requires click>=8.0, but you have click 7.1.2 which is incompatible.
pip-tools 6.13.0 requires click>=8, but you have click 7.1.2 which is incompatible.


In [14]:
%%writefile app.py
import pandas as pd
import streamlit as st
import json
import streamlit.components.v1 as components
import os
import openai
import getpass
import pandas as pd

# openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')
openai.api_key  = st.text_input("OpenAI API Key", '',type="password")

# st.write('# Moviepro.ai')
st.markdown("<h1 style='text-align: center; color: blue;'>Paraphrase Dataset Generator</h1>", unsafe_allow_html=True)
st.write("## Create paraphrase dataset, powered by GPT-3.")

st.write("")
if "original" not in st.session_state:
    st.session_state.original = ""
if "paraphrase" not in st.session_state:
    st.session_state.paraphrase = ""
if "df" not in st.session_state:
    st.session_state.df = pd.DataFrame(columns=['original','paraphrase'])

def getparaphrase():
  response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="I am a diverse paraphrasing expert. I will generate many random pairs of original and paraphrased sentences.\n\nHere are a few:\n\noriginal: Once, a group of frogs was roaming around the forest in search of water.\nparaphrase:  A gang of frogs was roaming about the woods in search of water once more.\n\noriginal:",
  temperature=0.7,
  max_tokens=100,
  top_p=0.9,
  frequency_penalty=0.5,
  presence_penalty=0.6,
  stop=["original"]
  )
  output_string = response['choices'][0]['text'].strip()
  output_modified = output_string.replace("paraphrase:","")
  original_sentence, paraphrased_sentence = output_modified.split('\n')
  st.session_state.original = original_sentence.strip()
  st.session_state.paraphrase = paraphrased_sentence.strip()

def addtodb():
  data = {
    'original': st.session_state.original,
    'paraphrase': st.session_state.paraphrase
    }

  st.session_state.df = st.session_state.df.append(data, ignore_index=True)
  st.write (st.session_state.df.shape)






st.button("Generate paraphrases",on_click=getparaphrase)
st.button("Add to Database",on_click=addtodb)

st.write("**Original:** "+st.session_state.original)
st.write("**Paraphrase:** "+st.session_state.paraphrase)
st.dataframe(st.session_state.df)

Writing app.py


In [18]:
!streamlit run app.py & npx localtunnel --port 8501

2023-06-01 12:14:36.968 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.142.213.118:8501


  A new version of Streamlit is available.

  See what's new at https://discuss.streamlit.io/c/announcements

  Enter the following command to upgrade:
  $ pip install streamlit --upgrade

npx: installed 22 in 3.883s
your url is: https://loose-files-sort.loca.lt
/content/app.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  st.session_state.df = st.session_state.df.append(data, ignore_index=True)
/content/app.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  st.session_state.df = st.session_state.df.append(data, ignore_index=True)
  Stopping...
^C


In [16]:
!wget -q -O - ipv4.icanhazip.com

34.142.213.118


In [17]:
!curl ipv4.icanhazip.com

34.142.213.118
